In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
import tensorflow.keras.backend as K

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [2]:
pmpm_ccm = pd.read_csv("../temp/pmpm_ccm.csv")
ref_ccm = pd.read_csv("../temp/ref.csv")

In [3]:
pmpm_ccm.columns = [col.upper() for col in pmpm_ccm.columns]
pmpm_ccm.rename(columns = {'PERSON_ID':'PERS_ID'}, inplace = True)

In [4]:
pmpm_ccm.columns

Index(['PERS_ID', 'MEMBER_ID', 'MYR', 'MBR_GNDR', 'AGE_AT_MIDMONTH',
       'ALLOW_AMT', 'ALLOW_IP', 'ALLOW_SNF', 'ALLOW_OP', 'ALLOW_ER',
       'ALLOW_RD', 'ALLOW_LAB', 'ALLOW_PATH', 'ALLOW_RX', 'ALLOW_UC',
       'ALLOW_PCP', 'ALLOW_SPEC', 'ALLOW_OVMHSA', 'ALLOW_HHDM', 'DAYS_IP',
       'DAYS_SNF', 'CC_HYPOTHYROID', 'CC_AMI', 'CC_ALZHEIMER',
       'CC_ALZHEIMER_PLUS', 'CC_ANEMIA', 'CC_ASTHMA', 'CC_ATRIAL_FIB',
       'CC_BPH', 'CC_CATARACT', 'CC_COPD', 'CC_DEPRESSION', 'CC_GLAUCOMA',
       'CC_HIP_FRACTURE', 'CC_HYPERLIPIDEMIA', 'CC_HYPERTENSION',
       'CC_OSTEOPOROSIS', 'CC_STROKE', 'CC_BREAST_CANCER',
       'CC_COLORECTAL_CANCER', 'CC_PROSTATE_CANCER', 'CC_LUNG_CANCER',
       'CC_ENDOMETRIAL_CANCER', 'CC_CHRONIC_KIDNEY', 'CC_DIABETES',
       'CC_HEART_FAILURE', 'CC_ISCHEMIC', 'CC_ARTHRITIS', 'CC_AUTISM',
       'CC_CEREBRAL_PALSY', 'CC_CYSTIC_FIBROSIS', 'CC_EPILEPSY',
       'CC_INT_DISAB', 'CC_LEARN_DISAB', 'CC_MOBILITY_IMPAIR',
       'CC_MULTIPLE_SCLEROSIS', 'CC_MUSCULAR_

In [5]:
drop_cols = ['MEMBER_ID', 'ALLOW_SNF', 'ALLOW_OP', 'ALLOW_RD', 'ALLOW_LAB', 'ALLOW_PATH', 'ALLOW_RX', 'ALLOW_UC',
       'ALLOW_PCP', 'ALLOW_SPEC', 'ALLOW_OVMHSA', 'ALLOW_HHDM', 'DAYS_IP', 'DAYS_SNF']

In [6]:
pmpm_ccm = pmpm_ccm.drop(columns = drop_cols, axis=1)

In [7]:
pmpm_ccm.MYR = pmpm_ccm.MYR.astype(int)
pmpm_ccm.PERS_ID = pmpm_ccm.PERS_ID.astype(str)
pmpm_ccm.AGE_AT_MIDMONTH = pmpm_ccm.AGE_AT_MIDMONTH.astype(int)
pmpm_ccm.ALLOW_AMT = pmpm_ccm.ALLOW_AMT.astype(int)
pmpm_ccm = pmpm_ccm.drop_duplicates(subset = ['PERS_ID', 'MYR']).reset_index(drop = True)

In [8]:
ref_ccm = ref_ccm[~ref_ccm.ref_create_date.isin(['Member (Self Referral)'])].reset_index(drop = True)
ref_ccm['REGISTER_DATE'] = pd.to_datetime(ref_ccm['ref_create_date'], format = '%Y-%m-%d')
ref_ccm.rename(columns = {'person_id':'PERS_ID'}, inplace = True)
ref_ccm.sort_values(by = ['PERS_ID', 'REGISTER_DATE'], inplace = True)
ref_ccm.dropna(axis = 0, subset = ['PERS_ID', 'REGISTER_DATE'], inplace = True)
ref_ccm = ref_ccm.groupby(by = 'PERS_ID').apply(lambda x: x.iloc[0]).reset_index(drop = True)

In [9]:
ref_ccm['REGISTER_DATE'] 

0       2014-02-12
1       2015-02-17
2       2014-10-22
3       2014-12-16
4       2019-11-21
           ...    
42045   2020-06-23
42046   2020-07-13
42047   2020-07-14
42048   2020-07-07
42049   2020-07-15
Name: REGISTER_DATE, Length: 42050, dtype: datetime64[ns]

In [10]:
pmpm_ccm.to_pickle('../temp/pmpm_ccm.p')
ref_ccm.to_pickle('../temp/ref_ccm.p')